In [1]:
import pandas as pd

train_features = pd.read_csv("./q3/RNA-Seq dataset/train.csv")
train_labels = pd.read_csv("./q3/RNA-Seq dataset/train_labels.csv")
test_featues = pd.read_csv("./q3/RNA-Seq dataset/test.csv")

In [3]:
print(train_features.head())
print()
print()
print(train_features.describe())
print()
print()
print(test_featues.info())

  Unnamed: 0  gene_0    gene_1    gene_2    gene_3     gene_4  gene_5  \
0   sample_0     0.0  2.017209  3.265527  5.478487  10.431999     0.0   
1   sample_1     0.0  0.592732  1.588421  7.586157   9.623011     0.0   
2   sample_2     0.0  3.511759  4.327199  6.881787   9.870730     0.0   
3   sample_5     0.0  3.467853  3.581918  6.620243   9.706829     0.0   
4   sample_6     0.0  1.224966  1.691177  6.572007   9.640511     0.0   

     gene_6    gene_7  gene_8  ...  gene_20521  gene_20522  gene_20523  \
0  7.175175  0.591871     0.0  ...    4.926711    8.210257    9.723516   
1  6.816049  0.000000     0.0  ...    4.593372    7.323865    9.740931   
2  6.972130  0.452595     0.0  ...    5.125213    8.127123   10.908640   
3  7.758510  0.000000     0.0  ...    5.726657    8.602588    9.928339   
4  6.754888  0.531868     0.0  ...    5.105904    7.927968    9.673966   

   gene_20524  gene_20525  gene_20526  gene_20527  gene_20528  gene_20529  \
0    7.220030    9.119813   12.003135  